In [1]:
import pandas as pd

diary = pd.read_csv("../data-sources/flightdiary.csv", header=1)

#Gathering filenames
diary["PathFile"] = diary.Date.str[2:] + "-" + diary.FlightNumber

# Splitting airport information
diary["FromICAO"] = diary.From.str[-5:-1]
diary["FromIATA"] = diary.From.str[-9:-6]
diary["FromCity"] = diary.From.str.split("/").str[0]
diary.FromCity = diary.FromCity.str[:-1]
diary.From = diary.From.str.split("/").str[1]
diary.From = diary.From.str[:-5]
diary["ToICAO"] = diary.To.str[-5:-1]
diary["ToIATA"] = diary.To.str[-9:-6]
diary["ToCity"] = diary.To.str.split("/").str[0]
diary.ToCity = diary.ToCity.str[:-1]
diary.To = diary.To.str.split("/").str[1]
diary.To = diary.To.str[:-5]

# Splitting airline information
diary["AirlineICAO"] = diary.Airline.str[-4:-1]
diary["AirlineIATA"] = diary.Airline.str[-7:-5]
diary.Airline = diary.Airline.str[:-9]
diary["AirlineColor"] = diary.AirlineICAO

# Prettifying names of cities, airports and airlines
diary.FromCity = diary.FromCity.str.replace("Istanbul", "İstanbul").str.replace("Izmir", "İzmir").str.replace("Antakya", "Hatay").str.replace("Ercan", "Nicosia").str.replace("Tromso", "Tromsø").str.replace("Sharm el Sheikh", "Sharm El-Sheikh")
diary.ToCity = diary.ToCity.str.replace("Istanbul", "İstanbul").str.replace("Izmir", "İzmir").str.replace("Antakya", "Hatay").str.replace("Ercan", "Nicosia").str.replace("Tromso", "Tromsø").str.replace("Sharm el Sheikh", "Sharm El-Sheikh")
diary.From = diary.From.str.replace("Ataturk", "Atatürk").str.replace("Sabiha Gokcen", "Sabiha Gökçen").str.replace("Sakirpasa", "Şakirpaşa").str.replace("Hatay", "Antakya").str.replace("Carsamba", "Çarşamba").str.replace("Esenboga", "Esenboğa").str.replace("Sharm el-Sheik Intl", "Sharm El-Sheikh").str.replace("Mersin Cukurova International Airport", "Çukurova").str.replace("Nikola Tesla Beograd", "Nikola Tesla").str.replace("MR Stefanik", "M. R. Štefánik")
diary.To = diary.To.str.replace("Ataturk", "Atatürk").str.replace("Sabiha Gokcen", "Sabiha Gökçen").str.replace("Sakirpasa", "Şakirpaşa").str.replace("Hatay", "Antakya").str.replace("Carsamba", "Çarşamba").str.replace("Esenboga", "Esenboğa").str.replace("Sharm el-Sheik Intl", "Sharm El-Sheikh").str.replace("Mersin Cukurova International Airport", "Çukurova").str.replace("Nikola Tesla Beograd", "Nikola Tesla").str.replace("MR Stefanik", "M. R. Štefánik")
diary.Airline = diary.Airline.str.replace("Transavia.com France","Transavia").replace("Norwegian Air International","Norwegian Air").replace("Pegasus Airlines","Pegasus").replace("AJet Airlines","AJet")
diary.AirlineColor = diary.AirlineICAO.str.replace("THY", "#ff0000").str.replace("PGT", "#ff8c00").str.replace("MSC", "#8000a4").str.replace("TKJ", "#00ABC9").str.replace("TVF", "#00a706").str.replace("AJA", "#216FFF").str.replace("SAS", "#0000FF").str.replace("SXS", "#F5DD00").str.replace("IBK", "#ff7171")


ad = diary.FromICAO.tolist()
for i in range(len(ad)):
    if ad[i][:2] in ["LT","HE"] or ad[i] == "LCEN": ad[i] = "03"
    elif ad[i][:2] in ["EF","LY","LI","LZ","LO"]: ad[i] = "02"
    elif ad[i][:2] in ["EN","ES","LF","EG"]: ad[i] = "01"
    else: ad[i] = "ERR"
diary["PortTZ"] = ad
diary.Departure = pd.to_datetime(diary.Date + " " + diary.Departure, utc=True).dt.strftime('%Y-%m-%d %H:%M:%S%z')
diary.Departure = pd.to_datetime(diary.Departure.str[:-4] + diary.PortTZ + ':' + diary.Departure.str[-2:], utc=True).apply(lambda x: x.timestamp()).astype(int)
diary.Duration = diary.Duration.str[:-3]
diary.Duration = ((diary.Duration.str[:-3].astype(int) * 60) + diary.Duration.str[-2:].astype(int)) * 60
diary.Arrival = diary.Departure + diary.Duration
diary.Duration = diary.Duration.astype(int)

diary.pop("PortTZ")

#Reordering columns
cols = diary.columns.tolist()
cols.insert(2, cols.pop(cols.index('FromCity')))
cols.insert(4, cols.pop(cols.index('FromICAO')))
cols.insert(5, cols.pop(cols.index('FromIATA')))
cols.insert(6, cols.pop(cols.index('ToIATA')))
cols.insert(7, cols.pop(cols.index('ToICAO')))
cols.insert(8, cols.pop(cols.index('ToCity')))
cols.insert(14, cols.pop(cols.index('AirlineICAO')))
cols.insert(15, cols.pop(cols.index('AirlineIATA')))
cols.pop(23)
cols.pop(23)
cols.pop(23)
cols.pop(23)
cols.insert(16, cols.pop(cols.index("AirlineColor")))
diary = diary[cols]
diary = diary.sort_values('Departure')


diary.to_csv("../data-sources/cleandiary.csv", index=False)